# EHS Resopnces

In [7]:
from sqlalchemy import create_engine
import pandas as pd
import matplotlib.pyplot as plt


1. **`from sqlalchemy import create_engine`**: Imports the **`create_engine`** function from **SQLAlchemy** to connect to a database.
2. **`import pandas as pd`**: Imports the **Pandas** library, which is a powerful tool for data manipulation and analysis.
3. **`import matplotlib.pyplot as plt`**: Imports the **`pyplot`** module from **Matplotlib** for creating visualizations like charts and graphs.


In [8]:
from sqlalchemy import create_engine


server_name = 'DESKTOP-0N0LQTG'
database_name = 'Nova_Scotia_Action_For_Health'


connection_string = f'mssql+pyodbc://{server_name}/{database_name}?trusted_connection=yes&driver=ODBC+Driver+17+for+SQL+Server'


engine = create_engine(connection_string)


try:
    with engine.connect() as conn:
        print("Connection successful!")
except Exception as e:
    print(f"Connection failed: {str(e)}")

Connection successful!


### Explanation

#### Imports
- **`from sqlalchemy import create_engine`**: Imports the `create_engine` function from sqlalchemy. This function is essential for creating a connection engine that allows Python to interact with a SQL database.

#### Variables
- **`server_name`**: `'DESKTOP-0N0LQTG'` 
- **`database_name`**: `'Nova_Scotia_Action_For_Health'`

#### Constructing Connection String
- Constructs a connection string using Python's f-string formatting:
  - `mssql+pyodbc://`: Specifies the dialect and driver to use (MSSQL with PyODBC driver).
  - `{server_name}/{database_name}`: Specifies the server and database to connect to.
  - `trusted_connection=yes`: Indicates Windows authentication should be used, where the current Windows credentials are used to authenticate.
  - `driver=ODBC+Driver+17+for+SQL+Server`: Specifies the ODBC driver to use for the connection. Adjust this according to the version of the ODBC driver installed on your system.

#### Creating the Engine
- Uses the `create_engine` function from sqlalchemy to create an engine (`engine`) object:
  - The `connection_string` is passed as an argument to `create_engine`, configuring the engine to connect to the specified SQL Server instance and database using the specified authentication method and driver.

#### Testing the Connection
- Attempts to establish a connection to the SQL Server database using the `engine.connect()` method:
  - If the connection is successful (`with` block), it prints "Connection successful!".
  - If an exception occurs during the connection attempt (e.g., authentication error, network issue), the `except` block catches the exception (`Exception as e`), and it prints "Connection failed: " followed by the error message (`str(e)`).


## Emergency Health Services Responses By Zone for 2022 and 2023

In [9]:
# SQL query
query = """

SELECT
    Zone,
    SUM(Actual) AS EHS_responses
    
FROM 
    dbo.Emergency_Health_Services
WHERE 
    Measure_Name = 'EHS Responses'
    AND DATEPART(YEAR, Date) IN (2022, 2023)
GROUP BY 
    Zone
ORDER BY 
    Zone;
"""


try:
    df = pd.read_sql(query, engine)
    print(df) 
except Exception as e:
    print(f"Error executing SQL query: {str(e)}")

       Zone  EHS_responses
0   Central        96099.0
1   Eastern        44840.0
2  Northern        48771.0
3   Western        57688.0


### Explanation

- **SQL Query**: This SQL query retrieves data from the `Emergency_Health_Services` table to calculate the total number of Emergency Health Services (EHS) responses (`Actual`) grouped by `Zone` for the years 2022 and 2023.

  - **Columns Selected**: 
    - Selects `Zone`.
    - Calculates the sum of `Actual` responses as `EHS_responses`.

  - **Filters**: 
    - Includes rows where `Measure_Name` is 'EHS Responses'.
    - Filters for `Date` years 2022 and 2023 using `DATEPART(YEAR, Date)`.

  - **Grouping**: 
    - Groups results by `Zone`.

  - **Ordering**: 
    - Orders the results by `Zone` for clarity and organization.

- **Error Handling**:
  - The `try` block attempts to execute the SQL query using Pandas `read_sql` function and prints the resulting DataFrame (`df`).
  - If an exception occurs during the execution (e.g., SQL query error), the `except` block catches the exception (`Exception as e`) and prints an error message indicating the issue (`Error executing SQL query: {str(e)}`).




## Average Emergency Health Services Response Times in Minutes by Zone for 2022 and 2023

In [10]:
# SQL query
query = """

SELECT 
    Zone,
    AVG(Actual) AS Average_Response_Time_Minutes
FROM 
    dbo.Emergency_Health_Services
WHERE 
    Measure_Name = 'EHS Response Times'
    AND DATEPART(YEAR, Date) IN (2022, 2023)
GROUP BY 
    Zone
ORDER BY 
    Zone;
"""


try:
    df = pd.read_sql(query, engine)
    print(df)  
except Exception as e:
    print(f"Error executing SQL query: {str(e)}")

       Zone  Average_Response_Time_Minutes
0   Central                    8569.214160
1   Eastern                    2677.314961
2  Northern                    4063.312871
3   Western                    4997.301816


### Explanation

- **SQL Query**: This SQL query retrieves data from the `Emergency_Health_Services` table to calculate the average response time (`Actual`) of Emergency Health Services (EHS) by `Zone` for the years 2022 and 2023.

  - **Columns Selected**: 
    - Selects `Zone`.
    - Calculates the average of `Actual` response times as `Average_Response_Time_Minutes`.

  - **Filters**: 
    - Includes rows where `Measure_Name` is 'EHS Response Times'.
    - Filters for `Date` years 2022 and 2023 using `DATEPART(YEAR, Date)`.

  - **Grouping**: 
    - Groups results by `Zone`.

  - **Ordering**: 
    - Orders the results by `Zone` for clarity and organization.

## YOY response Time

In [11]:
# SQL query
query = """

WITH YearlyResponseTimes AS (
    SELECT 
        Zone,
        YEAR(Date) AS Year,
        AVG(Actual) AS AvgResponseTime
    FROM dbo.Emergency_Health_Services
    WHERE Measure_Name = 'EHS Response Times'
    GROUP BY Zone, YEAR(Date)
),
PreviousYearResponseTimes AS (
    SELECT 
        Zone,
        Year,
        AvgResponseTime,
        LAG(AvgResponseTime) OVER (PARTITION BY Zone ORDER BY Year) AS PreviousYearResponseTime
    FROM YearlyResponseTimes
)
SELECT 
    Zone,
    Year,
    AvgResponseTime,
    PreviousYearResponseTime,
    CASE 
        WHEN PreviousYearResponseTime IS NULL THEN NULL
        ELSE ((AvgResponseTime - PreviousYearResponseTime) * 100.0 / PreviousYearResponseTime) 
    END AS YoY_Growth_Percentage
FROM PreviousYearResponseTimes
WHERE Year IN (2022, 2023)
ORDER BY Zone, Year;
"""


try:
    df = pd.read_sql(query, engine)
    display(df)  
except Exception as e:
    print(f"Error executing SQL query: {str(e)}")

,Zone,Year,AvgResponseTime,PreviousYearResponseTime,YoY_Growth_Percentage
0,Central,2022,8516.342122,4586.499023,85.682850
1,Central,2023,8618.019118,8516.342122,1.193905
2,Eastern,2022,2460.249247,1503.401385,63.645535
3,Eastern,2023,2877.683312,2460.249247,16.967145
4,Northern,2022,3908.258809,2308.598712,69.291388
5,Northern,2023,4206.439697,3908.258809,7.629507
6,Western,2022,4475.103780,2588.499225,72.884107
7,Western,2023,5479.330773,4475.103780,22.440306


### Explanation

- **SQL Query**: This SQL query calculates the year-over-year growth in average Emergency Health Services (EHS) response times (`Actual`) by `Zone` for the years 2022 and 2023.

  - **Common Table Expressions (CTEs)**:
    - **YearlyResponseTimes**: 
      - Selects `Zone`, `YEAR(Date)` as `Year`, and calculates the average of `Actual` response times as `AvgResponseTime`.
      - Filters for rows where `Measure_Name` is 'EHS Response Times'.
      - Groups results by `Zone` and `YEAR(Date)` to compute average response times per year.

    - **PreviousYearResponseTimes**:
      - Selects `Zone`, `Year`, `AvgResponseTime`, and uses `LAG()` function to retrieve the previous year's `AvgResponseTime` (`PreviousYearResponseTime`) within each `Zone`.
      - Orders by `Year` within each `Zone`.

  - **Main Query**:
    - Selects `Zone`, `Year`, `AvgResponseTime`, `PreviousYearResponseTime`, and calculates the year-over-year growth percentage (`YoY_Growth_Percentage`) using the formula:
      \[
      \text{YoY Growth Percentage} = \left( \frac{{\text{AvgResponseTime} - \text{PreviousYearResponseTime}}}{{\text{PreviousYearResponseTime}}} \right) \times 100
      \]
      - If `PreviousYearResponseTime` is `NULL`, sets `YoY_Growth_Percentage` to `NULL`.
    - Filters for years 2022 and 2023.
    - Orders results by `Zone` and `Year` for clarity and organization.

## Year over Year growth in EHS responses

In [12]:
# SQL query
query = """

WITH YearlyResponses AS (
    SELECT 
        Zone,
        YEAR(Date) AS Year,
        SUM(Actual) AS TotalResponses
    FROM dbo.Emergency_Health_Services
    WHERE Measure_Name = 'EHS Responses'
    GROUP BY Zone, YEAR(Date)
),
PreviousYearResponses AS (
    SELECT 
        Zone,
        Year,
        TotalResponses,
        LAG(TotalResponses) OVER (PARTITION BY Zone ORDER BY Year) AS PreviousYearResponses
    FROM YearlyResponses
)
SELECT 
    Zone,
    Year,
    TotalResponses,
    PreviousYearResponses,
    CASE 
        WHEN PreviousYearResponses IS NULL THEN NULL
        ELSE ((TotalResponses - PreviousYearResponses) * 100.0 / PreviousYearResponses) 
    END AS YoY_Growth_Percentage
FROM PreviousYearResponses
WHERE Year IN (2022, 2023)
ORDER BY Zone, Year;
"""


try:
    df = pd.read_sql(query, engine)
    display(df) 
except Exception as e:
    print(f"Error executing SQL query: {str(e)}")

,Zone,Year,TotalResponses,PreviousYearResponses,YoY_Growth_Percentage
0,Central,2022,49236.0,36655.0,34.322739
1,Central,2023,46863.0,49236.0,-4.819644
2,Eastern,2022,23309.0,17082.0,36.453577
3,Eastern,2023,21531.0,23309.0,-7.627955
4,Northern,2022,24962.0,19143.0,30.397534
5,Northern,2023,23809.0,24962.0,-4.619021
6,Western,2022,29504.0,22185.0,32.990760
7,Western,2023,28184.0,29504.0,-4.473970


### Common Table Expressions (CTEs):

**YearlyResponses:**
- Selects `Zone`, `YEAR(Date) AS Year`, and `SUM(Actual) AS TotalResponses`.
- Filters data where `Measure_Name` is 'EHS Responses'.
- Groups the results by `Zone` and `YEAR(Date)` to calculate the total responses per year.

**PreviousYearResponses:**
- Uses the `LAG()` window function to retrieve the total responses from the previous year for each zone (`PARTITION BY Zone ORDER BY Year`).
- This allows comparison between consecutive years to compute the year-over-year growth.

### Main Query:

- Selects `Zone`, `Year`, `TotalResponses`, `PreviousYearResponses`, and calculates `YoY_Growth_Percentage`.
- `YoY_Growth_Percentage` is computed as:
- This formula calculates the percentage change in total responses from the previous year to the current year.
- If `PreviousYearResponses` is NULL (for the earliest year in the dataset), the result is NULL.

### Filtering and Sorting:

- `WHERE Year IN (2022, 2023)`: Filters the results to include only data from the years 2022 and 2023.
- `ORDER BY Zone, Year`: Orders the results first by `Zone` and then by `Year` for clarity and consistency in presentation.

